## SVM and Random Forest Classifier

Here we will train two models, a Random Forest Model and a Support Vector Machine.
We will then combine and average their outcomes. 
Combining these two models will hopefully work to give us a much better accuracy for predicting fake news. Both models work in different ways;

- Decision Trees in a random forest are all trained independently on subsets of the data. Each tree looks at random parts of the text and decides wether it thinks the news is real or fake, then a final decision is reached by whatever the majority of trees decided.
    - RFs are less prone to outlier influence as its result depends on the outcomes of multiple trees
    - The randomness of feature selection, and of the data each tree trains on, helps ensure that the model will give more accurate predictions for unseen data 

- The Support Vector Machine will try and create the best decision boundary it can to separate real and fake news. If all news articles were plotted as dots on a graph, the SVM would try and fine the best line it can that separates the real ones from the fake ones, where the line is as far from any articles as it can be.
    - SVMs work well for classifying complex data, as data can be mapped in a higher dimension, making it easier to find a decision boundary (We can choose what kernel to use for the model, which decides how to form the decision boundary)
    - Also generalizes well to unseen data 

If we combine both methods we can combine the strengths of both models and potentially get more accurate predictions.

In [80]:
#Imports
import pandas as pd
import numpy as np
import pickle
import nltk
from nltk.corpus import stopwords 
from textblob import TextBlob
from textblob import Word
import re
from scipy.sparse import hstack

from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

#Import Vectorizers
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, HashingVectorizer

#TextPreprocess.py provides functions for preprocessing a piece of text and a dataset
from TextPreprocess import preprocessText, preprocessDataset

## Text Preprocessing

In [82]:
# Reading dataset from .csv file
dataset = pd.read_csv('Datasets/fake_or_real_news_with_sentiment.csv')

dataset.head()

,Unnamed: 0,title,text,label,sentiment
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE,1
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE,1
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL,1
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE,1
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL,1


In [83]:
# Preprocessing data

#preprocessDataset((dataset to process), (name of column with text to be processed))
preprocessedData = preprocessDataset(dataset, 'text')

In [84]:
preprocessedData.head()

0    daniel greenfield shillman journalism fellow f...
1    google pinterest digg linkedin reddit stumbleu...
2    u secretary state john f kerry said monday sto...
3    kaydee king kaydeeking november 9 2016 lesson ...
4    primary day new york frontrunners hillary clin...
Name: text, dtype: object

In [85]:
dataset.head()

,Unnamed: 0,title,text,label,sentiment
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE,1
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE,1
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL,1
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE,1
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL,1


## Vectorizing

We need to convert the raw text we want to train our model on into numbers that our model can understand.

#### <u>TF-IDF:</u>
(Term Frequency Inverse Document Frequency)
- How often words are found in a document and how unique they are to the document
- Contains information on the most and least important words in the document
- Purely based off occurrence of words, does not take into account positions of words in the text

####  <u>Bag of Words:</u>
(Count vectorizer)
- Counts occurrences of words in the document
- Similar to TF-IDF except doesnt look at importance of words

#### <u>Hashing Vectorizer:</u>
- Convert a document to a matrix of token occurrences.
- Uses hashing to map token strings to feature indexes

#### <u>N-Grams:</u>
- Combinations of all adjacent words of length N
- Counts the occurrences of these N-grams in the document
- Maintains word order, but there is a tradeoff with choosing a value for N. Smaller N may result in less useful information, a higher N will give however give a very large matrix with lots of features. May be too many features


In [86]:
#TFIDF Vectorizer
tfidf_vectorizer = TfidfVectorizer(stop_words= 'english', max_df= 0.7)

tfidf = tfidf_vectorizer.fit_transform(preprocessedData)

In [87]:
#Bag of Words Vectorizer
bow_vectorizer = CountVectorizer(stop_words= 'english')

bow= bow_vectorizer.fit_transform(preprocessedData)


In [88]:
#Hashing Vectorizer
hashing_vectorizer = HashingVectorizer(n_features= 2**20)

hashed = hashing_vectorizer.fit_transform(preprocessedData)


In [55]:
#N-Grams
#Used for tokenizing then using count vectorizer to turn the ngrams to numbers

ngram_range = (1,3)

ngramVectorizer = CountVectorizer(ngram_range=ngram_range)

ngram = ngramVectorizer.fit_transform(preprocessedData)

In [57]:
#N-Grams TF-IDF

ngram_range = (1,3)

ngram_TFIDF_Vec = TfidfVectorizer(ngram_range=ngram_range)

ngram_tfidf = ngram_TFIDF_Vec.fit_transform(preprocessedData)

In [89]:
# Separating labels from rest of dataset
labels = dataset.label

# Ensuring sentiment and text are combined properly
features = hstack((tfidf, np.array(dataset.sentiment).reshape(-1, 1)))

# Split data into training and testing
x_train, x_test, y_train, y_test = train_test_split(features, labels, test_size= 0.2, random_state= 7)

# Models

### SVM

In [90]:
#Setting up the SVM
# Probability set to true so we can use soft voting with the random forest
svm_model = svm.SVC(kernel='linear', probability=True)

### Random Forest

In [91]:
#Setting up the random forest model
rf = RandomForestClassifier()

### Combine Both models using voting
Allows us to get the majority output of both models and combine them into one classifier.

Soft voting predicts the label based on predicted probabilities.

**Takes a long time to train (2 - 3 minutes generally, possibly >11 with N-grams)**

In [92]:
Combined_Models = VotingClassifier(estimators=[('svm', svm_model), ('randomForest', rf)], voting='soft')

# Fit our training data to the ensemble model

Combined_Models = Combined_Models.fit(x_train, y_train)

## Prediction and Accuracy

Some sample accuracy scores (** There are other metrics for accuracy we should look at too **)
- Using TF_IDF = 0.934
- Using BOW = 0.89
- Using hashing = 0.927
- Using N-grams (N = 3) and BOW = 0.887 [more confidence in predictions than without n-grams]
- Using N-grams (N = 3) and TF-IDF = 0.922 [more confidence in predictions than without n-grams]


In [93]:
#predict labels for testing set 
y_pred_combined = Combined_Models.predict(x_test)

In [94]:
#accuracy_svm = accuracy_score(y_test, y_pred_svm)
#print("SVM Accuracy: ", accuracy_svm)
#accuracy_rf = accuracy_score(y_test, y_pred_rf)
#print("Random Forest Accuracy: ", accuracy_rf)

accuracy_combined = accuracy_score(y_test, y_pred_combined)
print("Combined Accuracy: ", accuracy_combined)

Combined Accuracy:  0.9321231254932912


## Test own Text

In [95]:
# Load the sentiment analysis model and vectorizer
sentiment_model = pickle.load(open('TrainedModels/SentimentAnalysisModel_V1.pkl', 'rb'))
sentiment_vectorizer = pickle.load(open('TrainedModels/sentimentVectorizer_V1.pkl', 'rb'))

In [96]:
#Sample text taken from RTE.ie
filePath = 'Test_Text.txt'

with open(filePath, 'r', encoding= 'utf-8') as file:
    sample_text = file.read()

#preprocessText(Text to be processed)
preprocessedText = preprocessText(sample_text)


#Get the sentiment score for the text
sentiment_sampleText = sentiment_vectorizer.transform([preprocessedText])
sentiment_Label = sentiment_model.predict(sentiment_sampleText)


#Vectorize the text
tfidf_sampleText = tfidf_vectorizer.transform([preprocessedText])


#Combine the features for prediction
features_ownText = hstack((tfidf_sampleText, sentiment_Label))


#Predict if the text is real or fake
samplePred_Combined = Combined_Models.predict(features_ownText)


#Confidence of the model in real/fake
Combined_confidence = Combined_Models.predict_proba(features_ownText)

print("Predicted Label Combined Models: ", samplePred_Combined)
print("Predicted Sentiment Combined Models (1 = negative, 5 = positive): ", sentiment_Label)
print("Model Confidence that article is Fake = ", Combined_confidence[0][0])
print("Model Confidence that article is Real = ", Combined_confidence[0][1])


Predicted Label Combined Models:  ['REAL']
Predicted Sentiment Combined Models (1 = negative, 5 = positive):  [1]
Model Confidence that article is Fake =  0.29120359008372665
Model Confidence that article is Real =  0.7087964099162734


## To Do:
- Refine models for better results
- Look into other metrics for calculating the accuracy
- Double check the Sentiment Model/Dataset (a majority of the predicted sentiments for the dataset are 1)
- Check if models are correctly being trained using sentiment. It doesn't seem like the sentiment score has much effect on the results yet